# Exploratory Data Analysis
The goal of this notebook is to explore the nba match details dataset. It has 47 columns, so there's quite a bit to unpack here. This notebook seeks to identify which features are important, and which ones are correlated. Trying to reduce our complexity here

# Step 0: Dependencies

## 0.1: Install dependencies
Since I am using a venv, we need to install these dependencies

In [2]:
!pip install pandas numpy

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/c5/89/ce1c7dc497f9a20644f6a7d2dd5bce6378a48321955178197fa3b55d6fe3/pandas-2.1.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/b7/db/4d37359e2c9cf8bf071c08b8a6f7374648a5ab2e76e2e22e3b808f81d507/numpy-1.25.2-cp310-cp310-win_amd64.whl.metadata
  Using cached numpy-1.25.2-cp310-cp310-win_amd64.whl.metadata (5.7 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/11.1 MB 9.6 MB/s eta 0:00:02
   -- ------------------------------------- 0.7/11.1 MB 9.2 MB/s eta 0:00:02
   ---- --------------

## 0.2: Import Dependencies
Now that we have installed our dependencies, we need to import them.

In [3]:
import pandas as pd
import numpy as np

## Step 1: Load data
Now, let's load in our data, and take a look at it. This dataset was a very lucky find, and has data from every single NBA game every up until 2022.

In [4]:
raw_df = pd.read_csv('NBA_MATCH_DETAILS.csv')

Let's take a look at the head to get an idea of what stats we are using.

In [5]:
raw_df.head()

,SEASON_ID,GAME_DATE,GAME_ID,MATCHUP,TEAM_NAME_HOME,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,...,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PTS_AWAY,PTS_HOME,WL_HOME
0,NaN,1981-03-29,28000934,ATL vs. MIL,Atlanta Hawks,1610612737,ATL,41.0,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,38.0,132,128,L
1,NaN,1981-03-26,28000919,ATL vs. IND,Atlanta Hawks,1610612737,ATL,43.0,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,20.0,115,107,L
2,NaN,1981-03-24,28000908,ATL vs. DET,Atlanta Hawks,1610612737,ATL,41.0,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,22.0,91,96,W
3,NaN,1981-03-14,28000845,ATL vs. CLE,Atlanta Hawks,1610612737,ATL,45.0,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,27.0,112,110,L
4,NaN,1981-03-13,28000839,ATL vs. DEN,Atlanta Hawks,1610612737,ATL,47.0,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,28.0,117,119,W


Looks pretty intimidating. Let's look at how many columns we have

In [6]:
raw_df.shape[1]

47

47 columns! This leaves us a lot of room to work with, but we definitely aren't going to want to fit this many columns into any sort of model